## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-24-10-42-10 +0000,nyt,China’s Tropical Resort Island Orders Evacuati...,https://www.nytimes.com/2025/08/24/world/asia/...
1,2025-08-24-10-39-23 +0000,bbc,Watch: Solar-powered cars start epic Australia...,https://www.bbc.com/news/videos/ckgd7r2mrl2o?a...
2,2025-08-24-10-30-00 +0000,wsj,AI’s Big Leaps Are Slowing. That Could Be a Go...,https://www.wsj.com/tech/ai/ais-big-leaps-are-...
3,2025-08-24-10-24-22 +0000,nypost,Russia accuses Kyiv of striking nuclear power ...,https://nypost.com/2025/08/24/world-news/russi...
4,2025-08-24-10-00-27 +0000,nypost,Zohran Mamdani wants to end all misdemeanor ch...,https://nypost.com/2025/08/24/us-news/nyc-mayo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,trump,24
29,ukraine,9
83,new,7
22,russia,7
63,california,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
125,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...,67
131,2025-08-23-12-29-00 +0000,wsj,The Texas Senate approved a redistricting plan...,https://www.wsj.com/politics/policy/texas-cong...,58
9,2025-08-24-10-00-00 +0000,latimes,California's lightning-fast push for partisan ...,https://www.latimes.com/politics/story/2025-08...,47
27,2025-08-24-07-02-31 +0000,nypost,Illinois Gov. Pritzker says Trump trying to ‘m...,https://nypost.com/2025/08/24/us-news/illinois...,44
38,2025-08-24-04-12-28 +0000,bbc,Trump's plan to send troops to Chicago is abus...,https://www.bbc.com/news/articles/cd7yperjxneo...,43


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
125,67,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...
3,40,2025-08-24-10-24-22 +0000,nypost,Russia accuses Kyiv of striking nuclear power ...,https://nypost.com/2025/08/24/world-news/russi...
51,30,2025-08-24-02-00-00 +0000,wsj,Bashing the president’s tariff policies could ...,https://www.wsj.com/politics/policy/gop-tariff...
55,28,2025-08-24-01-14-02 +0000,nypost,"Rikers Island inmate found dead in cell, marki...",https://nypost.com/2025/08/23/us-news/rikers-i...
129,27,2025-08-23-12-42-42 +0000,cbc,U.S. seeks to deport Kilmar Abrego Garcia to U...,https://www.cbc.ca/news/world/kilmar-abrego-ga...
64,26,2025-08-24-00-05-20 +0000,nypost,Zohran Mamdani fails at bench press while tryi...,https://nypost.com/2025/08/23/us-news/zohran-m...
103,24,2025-08-23-18-30-32 +0000,nypost,"North Korea builds secret military base, incre...",https://nypost.com/2025/08/23/world-news/north...
131,23,2025-08-23-12-29-00 +0000,wsj,The Texas Senate approved a redistricting plan...,https://www.wsj.com/politics/policy/texas-cong...
36,22,2025-08-24-04-53-54 +0000,nypost,Rhode Island prosecutor who told cops they’d ‘...,https://nypost.com/2025/08/24/us-news/rhode-is...
91,22,2025-08-23-20-15-36 +0000,nypost,"Putin war hero, the ‘Executioner,’ exposed as ...",https://nypost.com/2025/08/23/world-news/putin...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
